In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-apr-2021/train.csv
/kaggle/input/tabular-playground-series-apr-2021/test.csv


This kernel is just more explanied version of the kernel https://www.kaggle.com/pranjalverma08/tps-april-21-the-ann-approach-score-80-782 .

For me I liked this kernel and wanted to get the feel of what is going on.

In [40]:
# !pip uninstall scikit-learn 
!pip install scikit-learn==0.24.1

In [41]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.1.


In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import theano
import tensorflow
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

# Reading Data

In [43]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/train.csv")
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/test.csv")

Description of Data

1. <b>survival: Survival 0 = No, 1 = Yes
1. <b>  pclass: Ticket class1 = 1st, 2 = 2nd, 3 = 3rd
1. <b>  sex: Sex
1. <b>  Age: Age in years
1. <b> sibsp: # of siblings / spouses aboard the Titanic
1. <b>  parch: # of parents / children aboard the Titanic
1. <b>  ticket: Ticket number
1. <b>  fare: Passenger fare
1. <b> cabin: Cabin number
1. <b> embarked: Port of Embarkation </b>

In [44]:
summary = pd.DataFrame(train_df.dtypes, columns = ["Dtypes"])
summary["Null"] = train_df.isnull().sum()
summary["first"] = train_df.iloc[0]
summary["second"] = train_df.iloc[1]
summary["third"] = train_df.iloc[2]
summary["unique"] = train_df.nunique()
print(len(train_df))
print(len(test_df))

100000
100000


In [45]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,2,"Bell, Adele",female,62.00,0,0,PC 15008,14.86,D17243,C
99996,99996,0,2,"Brown, Herman",male,66.00,0,0,13273,11.15,NaN,S
99997,99997,0,3,"Childress, Charles",male,37.00,0,0,NaN,9.95,NaN,S
99998,99998,0,3,"Caughlin, Thomas",male,51.00,0,1,458654,30.92,NaN,S


We can see that there are total of 100k rows in both train and test data.

In [46]:
summary

,Dtypes,Null,first,second,third,unique
PassengerId,int64,0,0,1,2,100000
Survived,int64,0,1,0,0,2
Pclass,int64,0,1,3,3,3
Name,object,0,"Oconnor, Frankie","Bryan, Drew","Owens, Kenneth",92144
Sex,object,0,male,male,male,2
Age,float64,3292,NaN,NaN,0.33,173
SibSp,int64,0,2,0,1,7
Parch,int64,0,0,0,2,8
Ticket,object,4623,209245,27323,CA 457703,75331
Fare,float64,134,27.14,13.35,71.29,15935


There are few null values in Ticket till Embarked column. There are names which are repetetive.

In [47]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

# Feature Engineering

In [48]:
train_df['Survived'].value_counts()
# To get the idea how the data is dustributed. what if it's unbalanced.

0    57226
1    42774
Name: Survived, dtype: int64

### Checking the correlation

In [49]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# To get the idea of how the chance to survive is related to the sibsp, lower the number of sibsp higher the chance of survival but not really

train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# To get the chance of survival based on the sex, females have comparatively higher chances than men.

train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# To check the correlation between the parents/children to chances of survival.

train_df[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# To check the correlation between the Embarked and chances of survival.


,Embarked,Survived
0,C,0.751296
1,Q,0.596976
2,S,0.315294


### Making copy of original dataset and dropping the unnecessary columns.

In [50]:
train_copy = train_df.copy()# made a copy of train and test sets
test_copy = test_df.copy()

train = train_df.drop(['PassengerId', 'Ticket'], axis = 1) # Dropping the unncessary column
test = test_df.drop(['PassengerId', 'Ticket',], axis = 1)

combine = [train, test] # combining both training and test set.


### Filling Null values and modifying values

In [51]:
train['Cabin'].fillna('U', inplace=True) # filling the null values with U i,e undefined
train['Cabin'] = train['Cabin'].apply(lambda x: x[0]) # filling all the values with the first alphabet that appeared!

test['Cabin'].fillna('U', inplace=True)
test['Cabin'] = test['Cabin'].apply(lambda x: x[0])

train['Cabin'].unique()
for dataset in combine:
    dataset['Cabin'] = dataset['Cabin'].fillna('U')
    dataset['Cabin'] = dataset['Cabin'].apply(lambda x: x[0])

### Checking correlation and creating frequency table

In [52]:
pd.crosstab(train['Cabin'], train['Survived'])
# To get the frequency between cabin and survived
train[['Cabin', 'Survived']].groupby(['Cabin'], as_index = False).mean().sort_values(by = 'Survived', ascending = True)
# Get the correlarion between both and cabin and Survived columns
cabin_mapping = {"T": 0, "U": 1, "A": 2, "G": 3, "C": 4, "F": 5, "B": 6, "E": 7, "D": 8}

for dataset in combine:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)
    dataset['Cabin'] = dataset['Cabin'].fillna(0)

### Creating frequency table for title and sex column

In [53]:

for dataset in combine:
    dataset['Title'] = dataset['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())

pd.crosstab(train['Title'], train['Sex'])
  
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

# Imputing Missing Values


In [54]:
train.isnull().sum()
test.isnull().sum()

Pclass         0
Name           0
Sex            0
Age         3487
SibSp          0
Parch          0
Fare         133
Cabin          0
Embarked     277
Title          0
dtype: int64

In [55]:
train['Age'].fillna(train['Age'].dropna().median(), inplace=True) # With median
test['Age'].fillna(train['Age'].mean(), inplace = True)# With mean

test['Fare'].fillna(train['Fare'].dropna().median(), inplace = True)# With median
train['Fare'].fillna(train['Fare'].dropna().median(), inplace = True)

train['Embarked'].fillna('C', inplace = True)
test['Embarked'].fillna('C', inplace = True)

train.drop('Title',axis=1,inplace=True) # dropping the title column
test.drop('Title',axis=1,inplace=True) 

### Sanity check

In [56]:
train.isnull().sum()
test.isnull().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

### Discretizing into bins

In [57]:
train['AgeBand'] = pd.cut(train['Age'], 5)
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.00692, 17.464]",0.437358
1,"(17.464, 34.848]",0.341962
2,"(34.848, 52.232]",0.447088
3,"(52.232, 69.616]",0.485275
4,"(69.616, 87.0]",0.632806


### Categorising the Data

In [58]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 4
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 0

In [59]:
dataset.Age.unique()

array([1., 3., 2., 4., 0.])

In [60]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
1,2,0.516254
15,18,0.500000
2,3,0.482436
9,10,0.452229
11,12,0.450000
5,6,0.440345
10,11,0.414062
3,4,0.409194
0,1,0.404394
4,5,0.335294


In [61]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [62]:
train['FareBand'] = pd.qcut(train['Fare'], 4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
train = train.drop(['FareBand'], axis=1)
combine = [train, test]

In [63]:
train = train.drop(['AgeBand', 'Name', 'SibSp', 'Parch' ], axis = 1)
test = test.drop(['Name', 'SibSp', 'Parch'], axis = 1)

In [64]:
x_train = train.drop('Survived', axis = 1)
y_train = train.Survived

x_test = test

In [65]:
horizontal_stack = pd.concat([x_train, y_train], axis=1)
horizontal_stack.index+=1
train_df=horizontal_stack


In [66]:
x_test.index+=100000
test_df=x_test

In [67]:
test_df

,Pclass,Sex,Age,Fare,Cabin,Embarked,FamilySize
100000,3,0,1.0,3,1,0,1
100001,3,1,3.0,0,1,0,1
100002,1,1,1.0,3,6,1,1
100003,2,0,1.0,1,1,0,1
100004,1,1,1.0,2,6,1,3
...,...,...,...,...,...,...,...
199995,3,1,1.0,1,1,2,1
199996,1,0,3.0,3,1,0,2
199997,3,0,2.0,1,1,0,1
199998,1,1,3.0,2,6,1,4


<b>After so much of things that we did, at this point it is okay to get lost of what are we trying to acheive here, so I tried my way to describe what all we have done through up until now.

<b> I hope this is helpful to you as well.

1. So at first we look at the files we have at our disposal.
1. Then we imported all the libraries up until now.
1. We read the data
1. We summarized the data to get more familiar with data, with how many null values are there and what data types of values are there, we also looked at the number of unique values of each column.
1. Then we went through the steps of Feature Engineering where in 
1. We first had a look at whether our training var is skewed or normalized.
2. We checked the correlation between few columns to get more familiar with them.
3. We then made the copy of our original dataset.
4. We dropped unnecessary columns, i.e passengerId and Ticket.
5. We then imputed the null values with U for cabin and also modified its values.
6. Later we then checked the correlation of cabin with survived column.
7. The reason why we have a combine dataset because it will make things easier if we want to change something which will hold true for both Train and Test set.
8. We then mapped the name column with its first name and did the change for both train and test set.
9. We looked the frequency table between newly created Title column which holds the first name of the Name col and then look at the correlation between it and Sex column which did tell much information.
10. Categorised the Sex column
11. Imputed missing values for all the columns which has missing values. 
12. Dropped the Title column.
13. Discretized into bins for column Age and created a new column, AgeBand and checked its corerlation
14. Added a new feature name FamilySize and checked its correlation.
15. Categorised the Embarked feature.
16. Discretized the Fare feature and checked its correlation and dropped the Fareband feature created while discretizing.
17. Dropped the Name, Sibsp and Parch feature.
18. Get the target feature and remove it from train data.
19. Get the horizontal_Stack and then get the test_df 
20. Split the data

# Model Building

### Getting target variable and splitting the data

In [68]:
y=train_df['Survived']
X=train_df.drop('Survived', axis =1)

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=101, test_size=0.25)

In [69]:
def build_classifier(optimizer):
    classifier = Sequential()
    
    classifier.add(Dense(units=10,kernel_initializer='uniform',activation='relu',input_dim=7))
    classifier.add(Dropout(rate = 0.2))
    
    classifier.add(Dense(units=64,kernel_initializer='uniform',activation='relu'))
    classifier.add(Dropout(rate = 0.2))
    
    classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    
    #Optimizer and Compilation
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return classifier

In [70]:
%%time
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[50,100,200],
                  batch_size=[16,32,64])
grid = HalvingRandomSearchCV(classifier, param_grid, scoring='accuracy', cv=10, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:289: UserWarning: The total space of parameters 9 is smaller than n_iter=3750. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Epoch 1/200
2344/2344 [==============================] - 4s 1ms/step - loss: 0.5467 - accuracy: 0.7301
Epoch 2/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5074 - accuracy: 0.7662
Epoch 3/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5039 - accuracy: 0.7688
Epoch 4/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5058 - accuracy: 0.7669
Epoch 5/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5030 - accuracy: 0.7686
Epoch 6/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5044 - accuracy: 0.7675
Epoch 7/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5050 - accuracy: 0.7678
Epoch 8/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5002 - accuracy: 0.7696
Epoch 9/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5046 - accuracy: 0.7672
Epoch 10/200
2344/2344 [==============================] - 3s 1ms/step - l

In [71]:
print(grid.best_params_)
print(grid.best_score_)

{'optimizer': 'Adam', 'epochs': 200, 'batch_size': 32}
0.7944444444444445


In [72]:
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

classifier.fit(X_train,y_train)
preds = classifier.predict(test_df)

Epoch 1/200
2344/2344 [==============================] - 4s 1ms/step - loss: 0.5471 - accuracy: 0.7310
Epoch 2/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5071 - accuracy: 0.7684
Epoch 3/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5083 - accuracy: 0.7652
Epoch 4/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5059 - accuracy: 0.7642
Epoch 5/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5027 - accuracy: 0.7691
Epoch 6/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5036 - accuracy: 0.7679
Epoch 7/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5069 - accuracy: 0.7664
Epoch 8/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5070 - accuracy: 0.7654
Epoch 9/200
2344/2344 [==============================] - 3s 1ms/step - loss: 0.5037 - accuracy: 0.7673
Epoch 10/200
2344/2344 [==============================] - 3s 1ms/step - l

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [73]:
y_final = (preds > 0.5).astype(int).reshape(test_df.shape[0])

output = pd.DataFrame({'PassengerId': test_df.index, 'Survived': y_final})
output.to_csv('pred_ann.csv', index=False)